In [ ]:
from symao.turbolence import *

turbolenceFormulas = createTurbolenceFormulary()

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 1
plt.rcParams['grid.color'] = "#cccccc"

In [ ]:
mCalc = Integrator(np, np.float64, 'abs')

samples3l = [('f', 0.0001, 8.0, 10000, 'linear')]

display(turbolenceFormulas['phaseSpatialPowerSpectrumVonKarman0_f'])
turbolenceFormulas.plotFormula('phaseSpatialPowerSpectrumVonKarman0_f', {'r_0':0.1, 'L_0':16}, samples3l, mCalc, log=True)

display(turbolenceFormulas['phaseSpatialPowerSpectrumGeneric_f'])
turbolenceFormulas.plotFormula('phaseSpatialPowerSpectrumGeneric_f', {'r_0':0.1}, samples3l, mCalc, log=True)

samples5 = [('r', 0.01, 16.0, 10000, 'linear')]

display(turbolenceFormulas['phaseVarianceVonKarman0'])
turbolenceFormulas.plotFormula('phaseVarianceVonKarman0', {'r_0':0.1, 'L_0':16}, samples5, mCalc, log=False)

N = 10000000
a = 0.0001
b = 8.0
samples3l = [('f', a, b, N, 'linear')]
v1, v2 = turbolenceFormulas.evaluateFormula('phaseSpatialPowerSpectrumVonKarman0_ff', {'r_0':0.1, 'L_0':16}, samples3l,  mCalc)
print( np.sum(v2) * (b-a)/N * 2 * np.pi )

In [ ]:
from sympy import hankel_transform, inverse_hankel_transform

r = sp.symbols(r'r', positive=True)
L_0 = sp.symbols(r'L_0', positive=True)
r_0 = sp.symbols(r'r_0', positive=True)
r = sp.symbols(r'r', positive=True)
k = sp.symbols(r'k', positive=True)
f = sp.symbols(r'f', positive=True)
nu = sp.symbols(r'nu', positive=True)

psd = turbolenceFormulas['phaseSpatialPowerSpectrumVonKarman0_f'].rhs.subs({L_0:16, r_0:0.1})
display(psd)
cr_vk_h = hankel_transform(psd, f, r, 0)
display(cr_vk_h)
print(cr_vk_h.subs({r:0.0000001}).evalf()*2*np.pi)
lambda_PSD_VK = sp.lambdify( f, psd, 'numpy' )

In [ ]:
psd = turbolenceFormulas['phaseSpatialPowerSpectrumGeneric_f'].rhs.subs({r_0:0.1})
display(psd)
cr_g_h = hankel_transform(psd, f, r, 0)
display(cr_g_h)
print(cr_g_h.subs({r:0.0000001}).evalf()*2*np.pi)
lambda_PSD_G = sp.lambdify( f, psd, 'numpy' )

In [ ]:
from scipy import fft

def compute_covariance_from_PSD( alambda_PSD, P=8, Q=8, points=10000):
    fd = np.logspace(-P, Q, points)  # Input evaluation points
    fln = np.log(fd[1]/fd[0])        # Step size
    offset_f = fft.fhtoffset(fln, initial= -(P-Q)*np.log(10), mu=0, bias=0)
    rd = np.exp(offset_f)/fd[::-1]*(10**(P-Q))   # Output evaluation points
    PDS_f = alambda_PSD(fd) * fd
    fht = 2*np.pi * fft.fht(PDS_f, fln, mu=0, offset=offset_f, bias = 0)/rd
    return fht, rd

In [ ]:

fht1, rd1 = compute_covariance_from_PSD(lambda_PSD_VK)
fht2, rd2 = compute_covariance_from_PSD(lambda_PSD_G)

print(rd1[-1])
wl1 = 2300
wl2 = 6500
print('Variance in 0, numerical Hankel transform:', fht1[wl1])
plt.loglog(rd1[wl1:wl2]/(2*np.pi), fht1[wl1:wl2])

wl1 = 2300
wl2 = 6500
print('Variance in 0, numerical Hankel transform:', fht2[wl1])
plt.loglog(rd2[wl1:wl2]/(2*np.pi), fht2[wl1:wl2])
plt.grid(True)

In [ ]:


# variance_symbolic = variance_symbolic_rhs.subs({r:0})
# print(variance_symbolic)
display(turbolenceFormulas['varianceVonKarman0'])
print('Variance in 0, analytic covariance:', turbolenceFormulas['varianceVonKarman0'].subs({L_0:16, r_0:0.1}).evalf() )

print('Variance in 0, analytic, Hankel transform of PSD:', cr_vk_h.subs({L_0:16, r_0:0.1, r:0.0000001}).evalf() * 2 * np.pi)

#print(cr_vk_h.subs({L_0:1, r_0:1, r:0.000001}).evalf() * 2 * np.pi )

In [ ]:
display(turbolenceFormulas['phaseStructureFunctionVonKarman0'])

In [ ]:
display(turbolenceFormulas['phaseSpatialPowerSpectrumVonKarman0']) # non sto usando questa ma quella che ha anche l0

In [ ]:
NN = int(512)
screen_diameter = 8.0
rr0 = 0.1
outer_scale = 16.0
pixel_scale = screen_diameter/float(NN)
phase, PSD, del_f = ft_phase_screen0(turbolenceFormulas, rr0, NN, pixel_scale, outer_scale)
plt.figure(figsize = (12,12))
plt.imshow( phase, cmap='jet')
plt.show()
plt.figure(figsize = (12,12))
plt.imshow( np.log(PSD+1e-9), cmap='jet')
plt.show()

In [ ]:
Cf = turbolenceFormulas['phaseVarianceVonKarman0'].rhs
Df = turbolenceFormulas['phaseStructureFunctionVonKarman0'].rhs
Cf0 = Cf.evalf(0)
display( sp.S(2) * (Cf0 - Cf) )
display( Df )
ddiff = Df - sp.S(2) * (Cf0 - Cf) 
display(ddiff.evalf())
print(Df==2* (Cf0 - Cf))

In [ ]:
NN = int(512)
pixel_scale = 8.0/float(NN)
plt.imshow( ft_phase_screen(turbolenceFormulas, 8.2, NN, pixel_scale, 16, 0.005), cmap='jet')

In [ ]:
def seeing_to_r0(seeing, lamda=500.E-9):
    return 0.98*lamda/(seeing*np.pi/(180.*3600.))

In [ ]:
from scipy.special import gamma, kv

def find_allowed_size(mx_size):
    n = 0
    while (2 ** n + 1) < mx_size:
        n += 1
    mx_size = 2 ** n + 1
    return mx_size    

class PhaseScreen(object):
    def __init__(self, mx_size, pixel_scale, r0, L0, l0, xp=np, random_seed=None, stencil_length_factor=4):
        self.requested_mx_size = mx_size
        self.mx_size = find_allowed_size(mx_size)
        self.pixel_scale = pixel_scale
        self.r0 = r0
        self.L0 = L0
        self.l0 = l0
        self.xp = xp
        self.stencil_length_factor = stencil_length_factor
        self.stencil_length = stencil_length_factor * self.mx_size
        if random_seed is not None:
            self.xp.random.seed(random_seed)
        # Coordinate of Fried's "reference point" that stops the screen diverging
        self.reference_coord = (1, 1)
        self.set_stencil_coords()     
        self.setup()

    def append(self, arr, values, axis=None):
        arr = self.xp.asanyarray(arr)
        if axis is None:
            if arr.ndim != 1:
                arr = arr.ravel()
            values = self.xp.ravel(values)
            axis = arr.ndim-1
        return self.xp.concatenate((self.xp.asarray(arr), self.xp.asarray(values)), axis=axis)


    def phase_covariance(self, r, r0, L0):
        # Make sure everything is a float to avoid nasty surprises in division!
        if self.xp.__name__=='cupy':
            r = self.xp.asnumpy(r)
        r0 = float(r0)
        L0 = float(L0)
        # Get rid of any zeros
        r += 1e-40
        A = (L0 / r0) ** (5. / 3)
        B1 = (2 ** (-5. / 6)) * gamma(11. / 6) / (self.xp.pi ** (8. / 3))
        B2 = ((24. / 5) * gamma(6. / 5)) ** (5. / 6)
        C = (((2 * self.xp.pi * r) / L0) ** (5. / 6)) * kv(5. / 6, (2 * self.xp.pi * r) / L0)
        cov = A * B1 * B2 * C
        cov = self.xp.asarray(cov)
        return cov

    def set_stencil_coords(self):
        self.stencil = self.xp.zeros((self.stencil_length, self.mx_size))
        self.stencil[:4] = 1
        self.stencil_coords = self.xp.array(self.xp.where(self.stencil==1)).T
        self.stencil_positions = self.stencil_coords * self.pixel_scale
        self.n_stencils = len(self.stencil_coords)

    def setup(self):        
        # set X coords
        self.X_coords = self.xp.zeros((self.mx_size, 2))
        self.X_coords[:, 0] = -1
        self.X_coords[:, 1] = self.xp.arange(self.mx_size)
        self.X_positions = self.X_coords * self.pixel_scale
        
        # calc separations
        positions = self.append(self.stencil_positions, self.X_positions, axis=0)
        self.seperations = self.xp.zeros((len(positions), len(positions)))
        
        px, py = positions[:,0], positions[:,1]
        delta_x_gridA, delta_x_gridB = self.xp.meshgrid(px, px)        
        delta_y_gridA, delta_y_gridB = self.xp.meshgrid(py, py)        
        delta_x_grid = delta_x_gridA - delta_x_gridB     
        delta_y_grid = delta_y_gridA - delta_y_gridB     
        self.seperations = self.xp.sqrt(delta_x_grid ** 2 + delta_y_grid ** 2)

        # make cov_mats
        self.cov_mat = self.phase_covariance(self.seperations, self.r0, self.L0)
        self.cov_mat_zz = self.cov_mat[:self.n_stencils, :self.n_stencils]
        self.cov_mat_xx = self.cov_mat[self.n_stencils:, self.n_stencils:]
        self.cov_mat_zx = self.cov_mat[:self.n_stencils, self.n_stencils:]
        self.cov_mat_xz = self.cov_mat[self.n_stencils:, :self.n_stencils]
        # make A
        # Cholesky solve can fail - if so do brute force inversion
        if self.xp.__name__ == 'numpy':
            try:
                cf = scipy.linalg.cho_factor(self.cov_mat_zz)
                inv_cov_zz = scipy.linalg.cho_solve(cf, self.xp.identity(self.cov_mat_zz.shape[0]))
            except scipy.linalg.LinAlgError:
                raise scipy.linalg.LinAlgError("Could not invert Covariance Matrix to for A and B Matrices. Try with a larger pixel scale")
        else:
                cf = cupyx.scipy.linalg.lu_factor(self.cov_mat_zz)
                inv_cov_zz = cupyx.scipy.linalg.lu_solve(cf, self.xp.identity(self.cov_mat_zz.shape[0]))
            
        self.A_mat = self.cov_mat_xz.dot(inv_cov_zz)
        # make B
        # Can make initial BBt matrix first
        BBt = self.cov_mat_xx - self.A_mat.dot(self.cov_mat_zx)
        # Then do SVD to get B matrix
        u, W, ut = self.xp.linalg.svd(BBt)
        L_mat = self.xp.zeros((self.mx_size, self.mx_size))
        self.xp.filambdadiagonal(L_mat, self.xp.sqrt(W))
        # Now use sqrt(eigenvalues) to get B matrix
        self.B_mat = u.dot(L_mat)
        # make initial screen
        self._scrn = ft_phase_screen( turbolenceFormulas, self.r0, self.stencil_length, self.pixel_scale, self.L0, self.l0 )        
        self._scrn = self._scrn[:, :self.mx_size]
   
    def get_new_row(self):
        random_data = self.xp.random.normal(size=self.mx_size) / ((2*self.xp.pi)**2)
        if self.xp.__name__ == 'numpy':
            stencil_data = self.xp.asarray(self._scrn[self.stencil_coords[:, 0], self.stencil_coords[:, 1]])
        else:
            stencil_data = self.xp.asarray(self._scrn[cp.asnumpy(self.stencil_coords[:, 0]), cp.asnumpy(self.stencil_coords[:, 1])])
        new_row = self.A_mat.dot(stencil_data) + self.B_mat.dot(random_data)
        self.xp.reshape( new_row, (1, self.mx_size) )
        return self.xp.asarray(new_row)
    
    def add_row(self):
        new_row = self.get_new_row()
        self._scrn = self.append([new_row], self._scrn, axis=0)[:self.stencil_length, :self.mx_size]
        return self.xp.asarray(self.scrn)

    def __repr__(self):
        return str(self.scrn)

    @property
    def scrn(self):
        if self.xp.__name__ == 'numpy':
            return self._scrn[:self.requested_mx_size, :self.requested_mx_size]
        else:
            return cp.asnumpy(self._scrn[:self.requested_mx_size, :self.requested_mx_size])


In [ ]:
NN = 512*2
pixel_scale = 8.0/float(NN)
scrn = PhaseScreen(NN, pixel_scale, seeing_to_r0(0.8), 20, 0.005, np)

plt.imshow(scrn.scrn, cmap='jet')
plt.colorbar()
plt.show()
for i in range(NN//4):
    scrn.add_row()
    
plt.imshow(scrn.scrn, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
%%timeit     
scrn.add_row()

In [ ]:
NN = 512*2
pixel_scale = 8.0/float(NN)
scrn = PhaseScreen(NN, pixel_scale, seeing_to_r0(0.8), 20, 0.005, cp)

plt.imshow(scrn.scrn, cmap='jet')
plt.colorbar()
plt.show()

for i in range(NN//4):
    scrn.add_row()
    
plt.imshow(scrn.scrn, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
%%timeit     
scrn.add_row()

In [ ]:
# REAL stuff ends here!!!!

In [ ]:
NN = 512
pixel_scale = 50.0/float(NN)
psd2d = ft_PSD_phi(turbolenceFormulas, 8, NN, pixel_scale, 1.0, 1e1-10)
# psd2ds = np.fft.fftshift(psd2d)
#plt.imshow( psd2d, cmap='jet')
plt.xscale('linear')
plt.yscale('linear')
plt.plot(psd2d[0][:, int(NN//2)])
plt.show()

In [ ]:
eeq = turbolenceFormulas.getFormula('phaseStructureFunctionOrig_r')

free_syms = { symbol.name:symbol for symbol in eeq.rhs.free_symbols }    
k, r = sp.symbols('k r', positive=True)
# ft = sp.HankelTransform(sp.simplify(rh), free_syms['r'], k, 0)
#r = free_syms['r']
expr = 1/ k** (sp.S(11)/sp.S(6))
display(expr)
ft = sp.InverseHankelTransform( expr, k, r, sp.S(0))
iii = ft.rewrite(sp.Integral)
display(iii)
res = ft.doit(noconds=True) 
print(res)
display(res)

kk =33
k0 = kk+1
max_n = 1
while True:
    if 2 ** (max_n - 1) + 1 >= kk:
        max_n -= 1
        break
    max_n += 1
print(max_n)

In [ ]:
kk =33
print( int(np.ceil(np.log2(kk-1) )) )

In [ ]:
mx_size = 256
pixel_scale = 1
stencil_length_factor = 4
stencil_length = stencil_length_factor * mx_size
stencil = np.zeros((stencil_length, mx_size))
        
max_n = int(np.ceil(np.log2(mx_size-1) )) 

for n in range(0, max_n + 1):
    col = int((2 ** (n - 1)) + 1)
    n_points = (2 ** (max_n - n)) + 1
    coords = np.round(np.linspace(0, mx_size - 1, n_points)).astype('int64')
    stencil[col - 1][coords] = 1
# Now fill in tail of stencil
for n in range(1, stencil_length_factor + 1):
    col = n * mx_size - 1
    stencil[col, mx_size // 2] = 1

stencil = np.zeros((stencil_length, mx_size))

stencil[0:4] = 1
    
stencil_coords = np.array(np.where(stencil == 1)).T
stencil_positions = stencil_coords * pixel_scale
n_stencils = len(stencil_coords)

import matplotlib.pyplot as plt

%matplotlib inline

plt.scatter(stencil_positions[:,0], stencil_positions[:,1])
print(stencil_positions[0,0::2])
#fig, ax = plt.subplots(figsize=(20*2, 5*2))
#ax.imshow(stencil[0:128, :], interpolation='nearest')
